In [ ]:
import pandas as pd
import glob
from azure.storage.blob.blockblobservice import BlockBlobService
import matplotlib.pyplot as plt
import numpy as np

pd.__version__

## Data download

Die gesammelten Daten befinden sich in der Azure Cloud, im Azure Storage Blob Format. Dabei enhält jede Datei einen 10 sekündigen Ausschnitt der Daten im CSV Format.

In Folge werden die einzelnen Dateien heruntergeladen und in eine temporäre CSV Datei test.csv geschrieben.

In [ ]:
block_blob_service = BlockBlobService(account_name='activityprofiles', account_key='vx5JFWqh2ajuF6iQey3xj0o3BJ0o2EZaPxtS1ME28a/9y1+e06SYXnYWoMqhh4OaY5ZRf4wKmZXgJf3im0Bmuw==')

generator = block_blob_service.list_blobs('app')

fp = open('test.csv', 'bw')

for blob in generator:
    # Using `get_blob_to_bytes`
    b = block_blob_service.get_blob_to_bytes('app', blob.name)
    fp.write(b.content)
    # Or using `get_blob_to_stream`
    # service.get_blob_to_stream(container_name, blob.name, fp)

fp.flush()
fp.close()

## Data cleaning

In [ ]:
lengths = []
file = open("test.csv")
lines = file.readlines()
file.close()

for line in lines:
    elements = line.split(",")
    lengths.append(len(elements))

pd.Series(lengths).value_counts()

Nach dem Einlesen der temporären CSV-Datei, muss ein erstes Problem behandelt werden. Es existieren CSV Zeilen, welche die Dezimalstellen der Sensorwerte mit ',' anstatt '.' trennten. Da wir dieses Zeichen ebenfalls zur Trennung der CSV Einträge nutzten, existieren nun Zeilen mit 33 Elemente anstatt 18. Das Problem könnte durch Spracheinstellungen oder Einstellungen des Smartphone Herstellers verursacht worden sein. Da nur wenige Werte mit 32 Elementen existieren, werden diese ignoriert.

Das Problem wurde bei Daten des Android Gerätes von Flavio festgestellt. Um es zu beheben, werden die betroffenen Kommas in folgendem Code durch Punkte ersetzt.

In [ ]:
# convert 32 to 18 length
def bad_line_handler(bad_line):
    elements = bad_line[:3]
    for i in range(0, len(bad_line[3:]), 2):
        elements.append(f"{bad_line[i+3]}.{bad_line[i+4]}")
    return elements

lines_ignored = 0
lines_new = []

# convert 33 elements to 18, drop 32
for i, line in enumerate(lines):
    elements = lines[i].split(",")
    if(len(elements) == 18):
        lines_new.append(line)
    elif(len(elements) == 33):
        correct_line = bad_line_handler(elements)
        lines_new.append(str.join(",", correct_line))
    elif(len(elements) == 32):
        # other invalid format, ignore line
        lines_ignored += 1
        pass
    else:
        raise (f"ERROR: unknown error at line {i}")

print(f"{lines_ignored} Zeilen wurden ignoriert.")
print(f"{len(lines_new)} Zeilen sind nun verfügbar.")

Die reparierten CSV Daten werden nun wieder in die temporäre CSV Datei geschrieben, deren Inhalt wird ersetzt.

In [ ]:
file = open("test.csv", "w")
file.writelines(lines_new)
file.close()

del lines
del lines_new

Die reparierte CSV Datei wird nun wieder eingelesen, diesmal als Pandas DataFrame. Weiter wird der Header der Daten hinzugefügt.

In [ ]:
df = pd.read_csv('test.csv', index_col=False, header=None) #engine="python", on_bad_lines=bad_line_handler
df.columns = ['time','name','activity','acc_x','acc_y','acc_z','mag_x','mag_y','mag_z','gyr_x','gyr_y','gyr_z','ori_x','ori_y','ori_z','ori_w','lat','long']

print(df.shape)

In [ ]:
 # drop test entry
 df_clean = df.loc[df['time'] != 'Test04.03.2022 07:48:54.917', :]

Ein Eintrag musste entfernt werden, welcher ganz zu Beginn den Weg in die Datenbasis gefunden hat. Damit kann time in ein DateTime Objekt konvertiert werden. Dies nimmt bei dieser Anzahl Datenpunkten einige Zeit in Anspruch.

In [ ]:
# convert datetime format, yess this takes 5 min. :/
df_clean.loc[:, 'time'] = pd.to_datetime(df_clean.loc[:, 'time'] + "000") # format="%d.%m.%Y %H:%M.%S.%f"

Als Nächstes werden alle Daten von dem 03.03.2022 gelöscht. Vor diesem Datum hatte die App einen Bug, welcher die Sensordaten verfälschte. 

In [ ]:
# drop all data before 3.3.2022, because of incorrect data loading
df_clean = df_clean[df_clean['time'] > pd.to_datetime('03.03.2022 08:00:00')]

In [ ]:
np.array(df_clean['name'].unique())

In [ ]:
df_clean['name'].value_counts()[0:10]

Die Eigenschaft "name" wird in der App als Freitext gesetzt. Wie man hier sehen kann, sind dabei sehr spezielle Namen eingetragen worden. Da diese Werte nur sehr selten vorgekommen sind, werden diese im Anschluss entfernt.

In [ ]:
# drop invalid user
df_clean = df_clean[(df_clean['name'].isin(['Raphi', 'pascal', 'Flavio', 'Ronny', 'Simon']))]

Das neue Dataframe wird nun gespeichert.

In [ ]:
df_clean.to_csv("df_clean.csv")

del df

## Data visualization

In [ ]:
df_clean = pd.read_csv("df_clean.csv")
df_clean.shape

In [ ]:
df_clean['activity'].value_counts().plot(kind='bar', title='Sensor readings per Activity', xlabel='Activity', ylabel='Number of sensor readings in Dataframe', figsize=(15, 5))
plt.show()

In dieser Darstellung ist dargstellt, wie häufig die einzelnen Aktivitäten im Datensatz vorkommen. In Folge sind die Anzahl Einträge noch in numerischer Form dargestellt.

In [ ]:
df_clean['activity'].value_counts()

In [ ]:
n = np.arange(len(df_clean['activity'].unique()))

fig, ax = plt.subplots(figsize=(22, 5))
names = []
for i, name in enumerate(df_clean['name'].unique()):
    ax.bar(n + (i/len(df_clean['name'].unique())/2 - 0.25), df_clean.loc[df_clean['name'] == name, 'activity'].value_counts().reindex(df_clean['activity'].unique(), fill_value=0), width=0.1)
    names.append(name)

# minline: 2h/5=24min, 1440s per person, 20meas/sec -> 28'800 measurements
plt.xticks(n, list(df_clean['activity'].unique()), rotation=45)
plt.legend(names)
ax.hlines(y=28800, xmin=-0.5, xmax=6.5, colors='green')
ax.set_title("Sensor Readings per Acitivity per Person")
ax.set_xlabel("Activity")
ax.set_ylabel("Sensor Readings per Activity per Person")
#plt.legend(a[1].unique())
plt.plot()

In dieser Darstellung sind die gemessenen Sensor Readings pro Aktivität je Person dargestellt. Im Challangekonzept wurde definiert, dass jede Person mindestens 20 Minuten jeder Aktivität aufzuzeichnen soll. Diese Grenze ist als grüne horizontale Linie eingezeichnet und diente uns zur Überprüfung, ob genügend Daten gesammelt wurden.

Die Anforderung konnte in der Aktivität "Elevatoring" von drei Personen nicht erfüllt werden. Im Lift war häufig das Senden der Daten nicht möglich, da keine Internetverbindung bestand. Da das autmatische Resending der App nicht wirklich funktionierte, wurden die Daten häufig nicht gepseichert. Wir waren uns dieser Einschränkung bewusst und definierten mit den vorhandenen Daten weiterzuarbeiten.

### Datenexploration

In diesen Darstellungen werden die gemessenen Koordinaten des GPS Signals visualisiert.

In [ ]:
# coordinates visualization
fig, ax = plt.subplots(figsize=(15, 5))
ax.scatter(df_clean['lat'].values, df_clean['long'].values)
ax.set_title("Koordinatenwerte")
ax.set_xlabel("Breitengrade")
ax.set_ylabel("Längengrade")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(22, 5))
ax.plot(df_clean.loc[(df_clean['name'] == 'pascal') & (df_clean['activity'] == 'Stairway'), ['mag_x', 'mag_y', 'mag_z']])
ax.set_title("Magnetometer Streppensteigen über Zeit [Pascal]")
ax.set_ylabel("Sensorwert")
ax.set_xlabel("Index Messwert")
plt.show()

In dieser Darstellung sind Daten vom Treppensteigen des Magnetometers über die Zeit visualisiert. 

In [ ]:
fig, ax = plt.subplots(figsize=(22, 5))
ax.plot(df_clean.loc[(df_clean['name'] == 'pascal') & (df_clean['activity'] == 'Elevatoring'), ['mag_x', 'mag_y', 'mag_z']])
ax.set_title("Magnetometer Liftfahren über Zeit [Pascal]")
ax.set_ylabel("Sensorwert")
ax.set_xlabel("Index Messwert")
plt.show()

In dieser Darstellung sind Daten des Magnetometers vom Liftfahren visualisiert. Bemerkenswert ist der Unterschied der Bewegungsprofile.